In [ ]:
import numpy as np
import pandas
import urllib
from bs4 import BeautifulSoup as bs
import os
import re
from datetime import datetime
import time

os.chdir('/home/idies/workspace/Storage/raddick/raddick_folksonomy/data/secondpass')
print('ok')

# Load data file

In [ ]:
# Read raw data
inmusic_df = pandas.read_csv('cleaned_audio_data.csv')
# Use music_df for processing; move rows to output when scored
music_df = inmusic_df

# IDs are unique, we can use them as pandices IWW - but now let's use rownumber #music_df.groupby('ID').size().sort_values(ascending=False)
print('Datafile read: {0:,.0f} rows.'.format(len(inmusic_df)))

# rename columns
music_df.index.name = 'rownumber'
music_df = music_df.rename(columns={'ID': 'id', 'Source': 'source', 'Sample File Name': 'filename'})
print('Columns renamed')

music_df = music_df.assign(filetype = music_df['filename'].apply(lambda x: x.split('.')[-1].strip()))
music_df = music_df.assign(stem = music_df['filename'].apply(lambda x: x[0:x.rfind('.')]))

print('backing up...')
music_df_bk = music_df

print('ok')
#music_df

# Make columns and fix a few values by hand

In [ ]:
print('reading from backup...')
music_df = music_df_bk

music_df = music_df.assign(date_format = 'otros')
music_df = music_df.assign(date_string = np.nan)
music_df = music_df.assign(thedate = pandas.to_datetime(np.nan))

music_df = music_df.assign(theyear = np.nan)
music_df = music_df.assign(themonth = np.nan)
music_df = music_df.assign(theday = np.nan)

music_df.loc[music_df['stem'] == '101-Dyin\' to Know -Joanne Shaw Taylor-The Sage Gateshead 2017-0-11-07', 
            ['date_string', 'thedate', 'date_format']] = ['2017-0-11-07', pandas.to_datetime('2017-11-07'), 'byhand']
music_df.loc[music_df['stem'] == 'john2017-0-21d01t01', 
            ['date_string', 'theyear', 'date_format']] = ['2017-0-21', 2017, 'byhand']
music_df.loc[music_df['stem'] == 'Osvaldo Golijov Azul (2006) -cck BA 2017-0-07-01', 
            ['date_string', 'thedate', 'date_format']] = ['2017-0-07-01', pandas.to_datetime('2017-07-01'), 'byhand']
music_df.loc[music_df['stem'] == 'gd68-00-00MidnightHour', ['date_string', 'theyear', 'date_format']] = ['68-00-00', 1968, 'byhand']
music_df.loc[music_df['stem'] == 'gd66-11-00.FM.interview', ['date_string', 'theyear', 'themonth', 'date_format']] = ['66-11-00', 1968, 11, 'byhand']
music_df.loc[music_df['stem'] == 'hartford1986-10-0xCulpepperT01', ['date_string', 'theyear', 'themonth', 'date_format']] = ['1986-10-0x', 1986, 10, 'byhand']

music_df['stem'].loc[11925]
print('backing up...')
music_df_bk = music_df

music_df.groupby('date_format').size()

#music_df[music_df['date_format'] == 'byhand']
#print('ok')

# Find filenames with proper yyyy-mm-dd date format

In [ ]:
print('reading from backup...')
music_df = music_df_bk

yyyymmddfinder = re.compile('(19|20)[\d]{2}\-[\d]{2}\-[\d]{2}')

music_df.loc[
    (music_df['date_format'] == 'otros')
    & (music_df['stem'].apply(lambda x: re.search(yyyymmddfinder,x) != None))
, 'date_format'] = 'yyyy-mm-dd'

music_df.loc[
    (music_df['date_format'] == 'yyyy-mm-dd')
, 'date_string'] = music_df['stem'][(music_df['date_format'] == 'yyyy-mm-dd')
                                   ].apply(lambda x: 
                                           x[re.search(yyyymmddfinder,x).start():re.search(yyyymmddfinder,x).end()]
                                          )
music_df.loc[(music_df['date_format'] == 'yyyy-mm-dd'), 'thedate'] = pandas.to_datetime(music_df['date_string'], errors='coerce')

print('backing up...')
music_df_bk = music_df

music_df.groupby('date_format').size()

#len(music_df['thedate'].dropna())

In [ ]:
music_df[['stem', 'date_string', 'thedate']][music_df['date_format'] != 'otros'].sample(10)

# Find filenames with date format <code>[\d]{1,4}\-[\d]{1,2}\-[\d]{1,4}</code>

In [ ]:
print('reading from backup...')
music_df = music_df_bk

dashfinder = re.compile('[\d]{1,4}\-[\d]{1,2}\-[\d]{1,4}')

music_df.loc[
    (music_df['date_format'] == 'otros') & (music_df['stem'].apply(lambda x: re.search(dashfinder,x) != None))
, 'date_format'] = 'dashfinder'

music_df.loc[
    (music_df['date_format'] == 'dashfinder')
, 'date_string'] = music_df['stem'][(music_df['date_format'] == 'dashfinder')
                                   ].apply(lambda x: 
                                           x[re.search(dashfinder,x).start():re.search(dashfinder,x).end()]
                                          )

print('parsing dates...')

#music_df.loc[
#    (music_df['date_format'] == 'dashfinder')
#, ['thedate', 'date_format_2']] = parse_dates(music_df['date_string'][music_df['date_format'] == 'dashfinder'])

print('backing up...')
music_df_bk = music_df

print(music_df.groupby('date_format').size())

#music_df[['date_string','thedate']][music_df['date_format'] == 'dashfinder']

#music_df.loc[2298]

## Parse acutal dates from those filenames

In [ ]:
print('retrieving from backup...')
music_df = music_df_bk

dateredict = {}
dateredict['xx-xx-xxxx'] = re.compile('[\d]{2}\-[\d]{2}\-[\d]{4}')
dateredict['x-xx-xxxx'] = re.compile('[\d]{1}\-[\d]{2}\-[\d]{4}')
dateredict['xx-x-xxxx'] = re.compile('[\d]{2}\-[\d]{1}\-[\d]{4}')
dateredict['x-x-xxxx'] = re.compile('[\d]{1}\-[\d]{1}\-[\d]{4}')

dateredict['xxxx-xx-xx'] = re.compile('[\d]{4}\-[\d]{2}\-[\d]{2}')
dateredict['xxxx-x-xx'] = re.compile('[\d]{4}\-[\d]{1}\-[\d]{2}')
dateredict['xxxx-xx-x'] = re.compile('[\d]{4}\-[\d]{2}\-[\d]{1}')
dateredict['xxxx-x-x'] = re.compile('[\d]{4}\-[\d]{1}\-[\d]{1}')

dateredict['xx-xx-xx'] = re.compile('[\d]{2}\-[\d]{2}\-[\d]{2}')
dateredict['x-xx-xx'] = re.compile('[\d]{1}\-[\d]{2}\-[\d]{2}')
dateredict['xx-x-xx'] = re.compile('[\d]{2}\-[\d]{1}\-[\d]{2}')
dateredict['x-x-xx'] = re.compile('[\d]{1}\-[\d]{1}\-[\d]{2}')

dateredict['xx-x-xx'] = re.compile('[\d]{2}\-[\d]{1}\-[\d]{2}')
dateredict['xx-xx-x'] = re.compile('[\d]{2}\-[\d]{2}\-[\d]{1}')
dateredict['xx-x-x'] = re.compile('[\d]{2}\-[\d]{1}\-[\d]{1}')



dates_df = pandas.DataFrame()
dates_df = dates_df.assign(date_string = music_df['date_string'][music_df['date_format'] == 'dashfinder'])
dates_df = dates_df.assign(date_format_2 = np.nan)

for nameofthisre, thisre in dateredict.items():
    dates_df.loc[(dates_df['date_format_2'].isnull()) & (dates_df['date_string'].apply(lambda x: re.search(thisre, x) != None)), 'date_format_2'] = nameofthisre

dates_df = dates_df.assign(date_string = music_df['date_string'][music_df['date_format'] == 'dashfinder'])
dates_df = dates_df.assign(part1 = dates_df['date_string'].apply(lambda x: x.split('-')[0]))
dates_df = dates_df.assign(part2 = dates_df['date_string'].apply(lambda x: x.split('-')[1]))
dates_df = dates_df.assign(part3 = dates_df['date_string'].apply(lambda x: x.split('-')[2]))
dates_df['part1'] = dates_df['part1'].apply(lambda x: int(x))
dates_df['part2'] = dates_df['part2'].apply(lambda x: int(x))
dates_df['part3'] = dates_df['part3'].apply(lambda x: int(x))    
dates_df = dates_df.assign(year = np.nan)
dates_df = dates_df.assign(month = np.nan)
dates_df = dates_df.assign(day = np.nan)
dates_df = dates_df.assign(extra = np.nan)

# Do some of the rare ones by hand to save time    
dates_df.loc[dates_df['date_string'] == '97-3-8', ['year', 'month', 'day', 'date_format_2']] = [1997, 3, 8, 'yy-m-d']
dates_df.loc[dates_df['date_string'] == '11-5-1701', ['year', 'month', 'day', 'extra', 'date_format_2']] = [2017, 11, 5, '01', 'mm-d-yyee']
dates_df.loc[dates_df['date_string'] == '11-9-2017', ['year', 'month', 'day', 'date_format_2']] = [2017, 11, 9, 'mm-d-yyyy']
dates_df.loc[dates_df['date_string'] == '93-11-1', ['year', 'month', 'day', 'date_format_2']] = [1993, 11, 1, 'yy-mm-d']
dates_df.loc[dates_df['date_string'] == '02-05-9', ['year', 'month', 'day', 'date_format_2']] = [2002, 5, 9, 'yy-mm-d']
dates_df.loc[dates_df['date_string'] == '2005-4-4', ['year', 'month', 'day', 'date_format_2']] = [2005, 4, 4, 'yyyy-m-d']
dates_df.loc[dates_df['date_string'] == '1984-9-3', ['year', 'month', 'day', 'date_format_2']] = [1984, 9, 3, 'yyyy-m-d']
dates_df.loc[dates_df['date_string'] == '6-11-0401', ['year', 'month', 'day', 'extra', 'date_format_2']] = [2004, 6, 11, '01', 'm-d-yyee']
dates_df.loc[dates_df['date_string'] == '2-08-2014', ['year', 'month', 'day', 'date_format_2']] = [2014, 2, 8, 'm-dd-yyyy']
dates_df.loc[dates_df['date_string'] == '9-12-2007', ['year', 'month', 'day', 'date_format_2']] = [2007, 9, 12, 'm-dd-yyyy']
dates_df.loc[dates_df['date_string'] == '71-4-22', ['year', 'month', 'day', 'date_format_2']] = [1971, 4, 22, 'yy-m-dd']
dates_df.loc[dates_df['date_string'] == '11-2-17', ['year', 'month', 'day', 'date_format_2']] = [2017, 11, 2, 'mm-d-yy']
dates_df.loc[dates_df['date_string'] == '11-3-17', ['year', 'month', 'day', 'date_format_2']] = [2017, 11, 3, 'mm-d-yy']
dates_df.loc[dates_df['date_string'] == '5-3-7303', ['year', 'month', 'day', 'extra', 'date_format_2']] = [1973, 5, 3, '03', 'm-d-yyee']
dates_df.loc[dates_df['date_string'] == '8-9-1701', ['year', 'month', 'day', 'extra', 'date_format_2']] = [2017, 8, 9, '01', 'm-d-yyee']
dates_df.loc[dates_df['date_string'] == '9-9-2017', ['year', 'month', 'day', 'date_format_2']] = [2017, 9, 9, 'm-d-yyyy']
dates_df.loc[dates_df['date_string'] == '9-7-2017', ['year', 'month', 'day', 'date_format_2']] = [2017, 9, 7, 'm-d-yyyy']


dates_df.loc[dates_df['date_format_2'] == 'xxxx-xx-x', ['year', 'month', 'day', 'date_format_2']] = [dates_df['part1'][dates_df['date_format_2'] == 'xxxx-xx-x'], dates_df['part2'][dates_df['date_format_2'] == 'xxxx-xx-x'], dates_df['part3'][dates_df['date_format_2'] == 'xxxx-xx-x'], 'yyyy-mm-d']
dates_df.loc[(dates_df['date_format_2'] == 'xxxx-xx-xx') & dates_df['part1'].apply(lambda x: x % 100 > 18), ['year', 'month', 'day', 'extra', 'date_format_2']] = [dates_df['part1'][(dates_df['date_format_2'] == 'xxxx-xx-xx') & dates_df['part1'].apply(lambda x: x % 100 > 18)].apply(lambda x: (x % 100) + 1900), dates_df['part2'][(dates_df['date_format_2'] == 'xxxx-xx-xx') & dates_df['part1'].apply(lambda x: x % 100 > 18)], dates_df['part3'][(dates_df['date_format_2'] == 'xxxx-xx-xx') & dates_df['part1'].apply(lambda x: x % 100 > 18)], dates_df['part1'][(dates_df['date_format_2'] == 'xxxx-xx-xx') & dates_df['part1'].apply(lambda x: x % 100 > 18)].apply(lambda x: str(x)[0:2]), 'eeyy-mm-dd']
dates_df.loc[(dates_df['date_format_2'] == 'xxxx-xx-xx') & dates_df['part1'].apply(lambda x: x % 100 <= 18), ['year', 'month', 'day', 'extra', 'date_format_2']] = [dates_df['part1'][(dates_df['date_format_2'] == 'xxxx-xx-xx') & (dates_df['part1'].apply(lambda x: x % 100 <= 18))].apply(lambda x: (x % 100) + 2000), dates_df['part2'][(dates_df['date_format_2'] == 'xxxx-xx-xx') & (dates_df['part1'].apply(lambda x: x % 100 <= 18))], dates_df['part3'][(dates_df['date_format_2'] == 'xxxx-xx-xx') & (dates_df['part1'].apply(lambda x: x % 100 <= 18))], dates_df['part1'][(dates_df['date_format_2'] == 'xxxx-xx-xx') & (dates_df['part1'].apply(lambda x: x % 100 <= 18))].apply(lambda x: str(x)[0:2]), 'eeyy-mm-dd']
dates_df.loc[(dates_df['date_format_2'] == 'x-x-xx') & (dates_df['part3'] > 18), ['year', 'month', 'day', 'date_format_2']] = [dates_df['part3'][(dates_df['date_format_2'] == 'x-x-xx') & (dates_df['part3'] > 18)].apply(lambda x: x + 1900), dates_df['part1'][(dates_df['date_format_2'] == 'x-x-xx') & (dates_df['part3'] > 18)], dates_df['part2'][(dates_df['date_format_2'] == 'x-x-xx') & (dates_df['part3'] > 18)], 'm-d-yy']
dates_df.loc[(dates_df['date_format_2'] == 'x-x-xx') & (dates_df['part3'] <= 18), ['year', 'month', 'day', 'date_format_2']] = [dates_df['part3'][(dates_df['date_format_2'] == 'x-x-xx') & (dates_df['part3'] <= 18)].apply(lambda x: x + 2000), dates_df['part1'][(dates_df['date_format_2'] == 'x-x-xx') & (dates_df['part3'] <= 18)], dates_df['part2'][(dates_df['date_format_2'] == 'x-x-xx') & (dates_df['part3'] <= 18)], 'm-d-yy']

dates_df.loc[(dates_df['date_format_2'] == 'xxxx-x-xx'), ['year', 'month', 'day', 'date_format_2']] = [dates_df['part1'][(dates_df['date_format_2'] == 'xxxx-x-xx')], dates_df['part2'][(dates_df['date_format_2'] == 'xxxx-x-xx')], dates_df['part3'][(dates_df['date_format_2'] == 'xxxx-x-xx')], 'yyyy-m-dd']

dates_df.loc[(dates_df['date_format_2'] == 'xx-xx-xxxx') & (dates_df['part3'] < 1900), ['year', 'month', 'day', 'extra', 'date_format_2']] = [dates_df['part3'][(dates_df['date_format_2'] == 'xx-xx-xxxx') & (dates_df['part3'] < 1900)].apply(lambda x: int(np.floor(x / 100)) + 2000), dates_df['part1'][(dates_df['date_format_2'] == 'xx-xx-xxxx') & (dates_df['part3'] < 1900)], dates_df['part2'][(dates_df['date_format_2'] == 'xx-xx-xxxx') & (dates_df['part3'] < 1900)], dates_df['part3'][(dates_df['date_format_2'] == 'xx-xx-xxxx') & (dates_df['part3'] < 1900)].apply(lambda x: '{:02}'.format(x % 100)), 'mm-dd-yyee']
dates_df.loc[(dates_df['date_format_2'] == 'xx-xx-xxxx') & (dates_df['part3'] > 2018), ['year', 'month', 'day', 'extra', 'date_format_2']] = [dates_df['part3'][(dates_df['date_format_2'] == 'xx-xx-xxxx') & (dates_df['part3'] > 2018)].apply(lambda x: int(np.floor(x/100)) + 1900), dates_df['part1'][(dates_df['date_format_2'] == 'xx-xx-xxxx') & (dates_df['part3'] > 2018)], dates_df['part2'][(dates_df['date_format_2'] == 'xx-xx-xxxx') & (dates_df['part3'] > 2018)], dates_df['part3'][(dates_df['date_format_2'] == 'xx-xx-xxxx') & (dates_df['part3'] > 2018)].apply(lambda x: '{:02}'.format(x % 100)), 'mm-dd-yyee']
dates_df.loc[(dates_df['date_format_2'] == 'xx-xx-xxxx') & (dates_df['part1'] > 12), ['year', 'month', 'day', 'date_format_2']] = [dates_df['part3'][(dates_df['date_format_2'] == 'xx-xx-xxxx') & (dates_df['part1'] > 12)], dates_df['part2'][(dates_df['date_format_2'] == 'xx-xx-xxxx') & (dates_df['part1'] > 12)], dates_df['part1'][(dates_df['date_format_2'] == 'xx-xx-xxxx') & (dates_df['part1'] > 12)], 'dd-mm-yyyy']
dates_df.loc[dates_df['date_format_2'] == 'xx-xx-xxxx', ['year', 'month', 'day', 'date_format_2']] = [dates_df['part3'][dates_df['date_format_2'] == 'xx-xx-xxxx'], dates_df['part1'][dates_df['date_format_2'] == 'xx-xx-xxxx'], dates_df['part2'][dates_df['date_format_2'] == 'xx-xx-xxxx'], 'mm-dd-yyyy']

dates_df.loc[(dates_df['date_format_2'] == 'x-xx-xx') & (dates_df['part3'] > 18), ['year', 'month', 'day', 'date_format_2']] = [dates_df['part3'][(dates_df['date_format_2'] == 'x-xx-xx') & (dates_df['part3'] > 18)].apply(lambda x: x + 1900), dates_df['part1'][(dates_df['date_format_2'] == 'x-xx-xx') & (dates_df['part3'] > 18)], dates_df['part2'][(dates_df['date_format_2'] == 'x-xx-xx') & (dates_df['part3'] > 18)], 'm-dd-yy']
dates_df.loc[(dates_df['date_format_2'] == 'x-xx-xx') & (dates_df['part3'] <= 18), ['year', 'month', 'day', 'date_format_2']] = [dates_df['part3'][(dates_df['date_format_2'] == 'x-xx-xx') & (dates_df['part3'] <= 18)].apply(lambda x: x + 2000), dates_df['part1'][(dates_df['date_format_2'] == 'x-xx-xx') & (dates_df['part3'] <= 18)], dates_df['part2'][(dates_df['date_format_2'] == 'x-xx-xx') & (dates_df['part3'] <= 18)], 'm-dd-yy']

dates_df.loc[(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part1'] > 31), ['year', 'month', 'day', 'date_format_2']] = [dates_df['part1'][(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part1'] > 31)].apply(lambda x: x + 1900), dates_df['part2'][(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part1'] > 31)], dates_df['part3'][(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part1'] > 31)], 'yy-mm-dd']
dates_df.loc[(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part3'] > 31),  ['year', 'month', 'day', 'date_format_2']] = [dates_df['part3'][(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part3'] > 31)].apply(lambda x: x + 1900), dates_df['part1'][(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part3'] > 31)], dates_df['part2'][(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part3'] > 31)], 'mm-dd-yy']
dates_df.loc[(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part1'] > 18), ['year', 'month', 'day', 'date_format_2']] = [dates_df['part3'][(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part1'] > 18)].apply(lambda x: x + 2000), dates_df['part2'][(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part1'] > 18)], dates_df['part1'][(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part1'] > 18)], 'dd-mm-yy']
dates_df.loc[(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part3'] > 18), ['year', 'month', 'day', 'date_format_2']] = [dates_df['part1'][(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part3'] > 18)].apply(lambda x: x + 2000), dates_df['part2'][(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part3'] > 18)], dates_df['part3'][(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part3'] > 18)], 'yy-mm-dd']
dates_df.loc[(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part2'] > 12), ['year', 'month', 'day', 'date_format_2']] = [dates_df['part3'][(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part2'] > 12)].apply(lambda x: x + 2000), dates_df['part1'][(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part2'] > 12)], dates_df['part2'][(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part2'] > 12)], 'mm-dd-yy']
dates_df.loc[(dates_df['date_format_2'] == 'xx-xx-xx'), ['year', 'month', 'day', 'date_format_2']] = [dates_df['part1'][(dates_df['date_format_2'] == 'xx-xx-xx')].apply(lambda x: x + 2000), dates_df['part2'][(dates_df['date_format_2'] == 'xx-xx-xx')], dates_df['part3'][(dates_df['date_format_2'] == 'xx-xx-xx')], 'yy-mm-dd']

#print(dates_df.groupby('date_format_2').size().sort_values(ascending=False))

dates_df = dates_df.assign(thedate=pandas.to_datetime(dates_df[['year', 'month', 'day']], errors='coerce'))

music_df.loc[dates_df.index, 'thedate'] = dates_df['thedate']
music_df.loc[dates_df.index, 'date_format'] = dates_df['date_format_2']

print('backing up...')
music_df_bk = music_df
#music_df[['date_string', 'thedate', 'date_format']].dropna().sample(20)
music_df.groupby('date_format').size()


## Get year/month/day individual values from full dates

In [ ]:
print('retrieving from backup...')
music_df = music_df_bk


music_df.loc[(music_df['thedate'].notnull()), 'theyear'] = music_df['thedate'][(music_df['thedate'].notnull())].apply(lambda x: x.year)
music_df.loc[(music_df['thedate'].notnull()), 'themonth'] = music_df['thedate'][(music_df['thedate'].notnull())].apply(lambda x: x.month)
music_df.loc[(music_df['thedate'].notnull()), 'theday'] = music_df['thedate'][(music_df['thedate'].notnull())].apply(lambda x: x.day)

print('backing up...')
music_df_bk = music_df

music_df.sample(10)

In [ ]:
print('retrieving from backup...')
music_df = music_df_bk

redict = {}
redict['longmonthname_day_year'] = re.compile('([j|J]anuary|[f|F]ebruary|[m|M]arch|[a|A]pril|[m|M]ay|[j|J]une|[j|J]uly|[a|A]ugust|[s|S]eptember|[o|O]ctober|[n|N]ovember|[d|D]ecember)[,\.\-\s]*[\d]+[st|nd|rd|th]*[,\.\-\s]*[\d]{4}')

for nameofthisre, thisre in redict.items():
    print(nameofthisre)
    #print(music_df['stem'][(music_df['date_format'] == 'otros')])
    music_df.loc[(music_df['date_format'] == 'otros') & (music_df['stem'].apply(lambda x: re.search(thisre, x) != None))
                 , 'date_string'] = music_df['stem'][
        (music_df['date_format'] == 'otros') 
        & (music_df['stem'].apply(lambda x: re.search(thisre, x) != None))
        ].apply(lambda x: x[re.search(thisre, x).start():re.search(thisre,x).end()])

#    print(
#        music_df['date_string'][
#            (music_df['date_string'].notnull()) & 
#            (music_df['date_string'].apply(lambda x: re.search(thisre, str(x)) != None))
#        ])

music_df['date_string'][(music_df['date_format'] == 'otros') & (music_df['date_string'].notnull())]

In [ ]:
print('retrieving from backup...')
music_df = music_df_bk

seps = re.compile('[,_\.\-\s]+')
ordinalios = re.compile('(st|nd|rd|th)*[\,]*[\s]+')

redict = {}
redict['longmonthname_day_year'] = re.compile('([j|J]anuary|[f|F]ebruary|[m|M]arch|[a|A]pril|[m|M]ay|[j|J]une|[j|J]uly|[a|A]ugust|[s|S]eptember|[o|O]ctober|[n|N]ovember|[d|D]ecember)[,\.\-\s]*[\d]+[st|nd|rd|th]*[,\.\-\s]*[\d]{2,4}')
redict['shortmonthname_day_year'] = re.compile('([j|J]an|[f|F]eb|[m|M]ar|[a|A]pr|[m|M]ay|[j|j]un|[j|J]ul|[a|A]ug|[s|S]ep|[o|O]ct|[n|N]ov|[d|D]ec)(?![a-z])[_,\.\-\s]*[\d]+[st|nd|rd|th]*[,\.\-\s]*[\d]{2,4}')
redict['day_longmonthname_year'] = re.compile('[\d]+(st|nd|rd|th)*[,\.\-\s]*([j|J]anuary|[f|F]ebruary|[m|M]arch|[a|A]pril|[m|M]ay|[j|j]une|[j|J]uly|[a|A]ugust|[s|S]eptember|[o|O]ctober|[n|N]ovember|[d|D]ecember)(?![a-z])[,\.\-\s]*[\d]{2,4}')
redict['day_shortmonthname_year'] = re.compile('[\d]+(st|nd|rd|th)*[,\.\-\s]*([j|J]an|[f|F]eb|[m|M]ar|[a|A]pr|[m|M]ay|[j|j]un|[j|J]ul|[a|A]ug|[s|S]ep|[o|O]ct|[n|N]ov|[d|D]ec)(?![a-z])[_,\.\-\s]*[\d]{2,4}')
redict['year_longmonthname_day'] = re.compile('([\d]+[,\.\-\s]*[j|J]anuary|[f|F]ebruary|[m|M]arch|[a|A]pril|[m|M]ay|[j|j]une|[j|J]uly|[a|A]ugust|[s|S]eptember|[o|O]ctober|[n|N]ovember|[d|D]ecember)(?![a-z])[,\.\-\s]*[\d]+[st|nd|rd|th]*')
redict['year_shortmonthname_day'] = re.compile('([\d]+[,\.\-\s]*[j|J]an|[f|F]eb|[m|M]ar|[a|A]pr|[m|M]ay|[j|j]un|[j|J]ul|[a|A]ug|[s|S]ep|[o|O]ct|[n|N]ov|[d|D]ec)(?![a-z])[,\.\-\s]*[\d]+[st|nd|rd|th]*')
redict['eightstraight'] = re.compile('[\d]{8}')
redict['sixstraight'] = re.compile('[\d]{6}')
redict['other_numeric'] = re.compile('[\d]{1,4}[,_\.\-\s]+[\d]{1,4}[,_\.\-\s]+[\d]{1,4}')
redict['just_year_month'] = re.compile('(19|20)[\d]{2}[,_\.\-\s]+[\d]{1,2}')
redict['just_year'] = re.compile('(19|20)[\d]{2}')

keys_to_remove = []

for nameofthisre, thisre in redict.items():
    music_df.loc[
        (music_df['date_format'] == 'otros')
        & (music_df['stem'].apply(lambda x: re.search(thisre,x) != None))
    , 'date_format'] = nameofthisre
    music_df.loc[
        (music_df['date_format'] == nameofthisre)
    , 'date_string'] = music_df['stem'][
        (music_df['date_format'] == nameofthisre)
    ].apply(lambda x: x[re.search(thisre,x).start():re.search(thisre,x).end()])
    if (len(music_df[music_df['date_format'] == nameofthisre]) > 0):
        print('Regex {0:}: found {1:,.0f} matching filenames...'.format(nameofthisre, len(music_df[music_df['date_format'] == nameofthisre])))
    else:
        keys_to_remove.append(nameofthisre)
        
for x in keys_to_remove:
    del redict[x]

#print('backing up...')
#music_df_bk = music_df

music_df.groupby('date_format').size()

#music_df[music_df['date_string'] == '000225']



## Turn date strings found above into real dates

In [ ]:
for x in redict.keys():
    print(x)
    
music_df[['date_string', 'thedate']][music_df['date_format'] == 'longmonthname_day_year']